In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [139]:
# Load cross-attention dataset
xatt_df = pd.read_csv("cross_attention_game_features.csv")
xatt_df['week'] = xatt_df['week'].astype(int)
weeks = sorted(xatt_df['week'].unique())
print("Weeks found in CSV:", weeks)

Weeks found in CSV: [np.int64(202401), np.int64(202402), np.int64(202403), np.int64(202404), np.int64(202405), np.int64(202406), np.int64(202407), np.int64(202408), np.int64(202409), np.int64(202410), np.int64(202411), np.int64(202412), np.int64(202413), np.int64(202414), np.int64(202415), np.int64(202416), np.int64(202417), np.int64(202418)]


In [140]:
# Features/labels
meta_cols = ['week', 'away', 'home', 'label']
X_all = xatt_df.drop(columns=meta_cols).values
y_all = xatt_df['label'].values
games_meta = xatt_df[['week','away','home']].copy()

In [264]:
# Helper to build fresh model each step
def build_interaction_model(input_dim=112):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [265]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,            # stop if no improvement for 7 epochs
    restore_best_weights=True
)

lr_sched = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,            # reduce LR by half
    patience=7,            # wait 5 epochs before reducing
    min_lr=1e-7,           # don’t go below this
    verbose=1
)

In [266]:
import warnings

warnings.filterwarnings('ignore')

results = []

for i in range(1, len(weeks)):
    wk = weeks[i]
    prev_wks = weeks[:i]

    train_idx = xatt_df['week'].isin(prev_wks)
    test_idx  = xatt_df['week'] == wk

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_test, y_test   = X_all[test_idx],  y_all[test_idx]

    print(f"Training weeks: {prev_wks[0]} → {prev_wks[-1]}, "
          f"Testing week: {wk}, "
          f"n_train={len(y_train)}, n_test={len(y_test)}")

    # --- Safeguard ---
    if len(y_test) == 0:
        print(f"⚠️ No test games for week {wk}, skipping…")
        continue

    model = build_interaction_model(input_dim=X_all.shape[1])

    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=200,
        batch_size=min(32, max(4, len(X_train)//16)),
        verbose=0,
        callbacks=[early_stop, lr_sched]
    )

    y_pred_prob = model.predict(X_test, verbose=0).flatten()
    y_pred = (y_pred_prob >= 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    ll  = log_loss(y_test, y_pred_prob, labels=[0,1])

    results.append({
        "week": wk,
        "n_games": len(y_test),
        "accuracy": acc,
        "log_loss": ll,
        "epochs_run": len(history.history['loss'])
    })

results_df = pd.DataFrame(results)
print(results_df)

Training weeks: 202401 → 202401, Testing week: 202402, n_train=16, n_test=16
Training weeks: 202401 → 202402, Testing week: 202403, n_train=32, n_test=16

Epoch 82: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 126: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Training weeks: 202401 → 202403, Testing week: 202404, n_train=48, n_test=16

Epoch 77: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Training weeks: 202401 → 202404, Testing week: 202405, n_train=64, n_test=14

Epoch 96: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 114: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Training weeks: 202401 → 202405, Testing week: 202406, n_train=78, n_test=14

Epoch 8: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Training weeks: 202401 → 202406, Testing week: 202407, n_train=92, n_test=15

Epoch 79: ReduceLROnPlateau reducing learning rate to 4.999999873689

In [267]:
model.save("XAtt_interaction_model.keras")